## TLE Parser

### Domain Specific Packaged Used

* https://pypi.org/project/ephem/ 
* https://federicostra.github.io/tletools/ 

### Use
This notebook includes various functions to validate and or correct the Air Force TLE data and produce gridded data capable of ingesting into the compute engine

This notebook assumes that data is stored at the following locations relative to this notebook
- TLE data to be validated is stored at `../data/TLE/source`
- TLEs that cant be validated are witten directly to a file at `../data/TLE/errors`
- Validated TLE data is written at `../data/TLE/processed`



Valid data is written in CSV format using the following schema 

| Norad ID | Epoch Year | Epoch JD | TLE |   |
|----------|------------|----------|-----|---|
| INT      | INT        | FLOAT    | STR |   |


In [ ]:
import pandas as pd
import ephem
import pathlib
from tletools.tle import TLE

In [ ]:
WorkingFolder = pathlib.Path.cwd().joinpath('../data/TLE')
source_files = list(WorkingFolder.joinpath('source').glob('*.txt'))
error_filess = list(WorkingFolder.joinpath('errors').glob('*.e'))
processed_files = list(WorkingFolder.joinpath('processed').glob('*.csv'))

In [ ]:
def initial_validate_tle_files(files):
    """
        This function takes a list of TLE files and validates each TLE in them using the Pyephem and TLE-tools library, 
        TLEs that can't be validated are written back to an error file directory as `<inputfilename>.e` for further processing
    """
    
    for file in files:
        print(f'processing {file.name}')
        with open(file) as tle_file:
            with open(WorkingFolder.joinpath(f'errors/{file.name[:-4]}.e'), 'w+') as error_file:
                i = 0
                tracks = []

                while True:
                    # try to create a pyephem and TLE-tools objects, if fail write TLE string to error file and proceed to next TLE
                    try:
                        # print to see progress
                        i += 1
                        if i % 500000 == 0:
                            print(i)

                        # compose TLE string to process in TLE-Tool object    
                        name = 'None\n'
                        firstline = tle_file.readline().replace('\\', '') # extra slash at the end of line one can cause a parsing error
                        secondline = tle_file.readline()

                        # detect if EOF
                        if firstline == '':
                            break

                        # run through pyephem for checksum detection 
                        ephem.readtle(name, firstline, secondline)

                        #run through tle-tools for easy accessing norad_id, epoch year, epoch jd
                        track = TLE.from_lines(name, firstline.strip(), secondline.strip()).asdict()
                        trackdict = {
                            'norad_id': track['norad'],
                            'epoch_year': track['epoch_year'],
                            'epoch_day': track['epoch_day'],
                            'tle': f"""{name}{firstline}{secondline}"""
                        }
                        tracks.append(trackdict)

                    except:
                        # write error
                        error_file.write(firstline)
                        error_file.write(secondline)

            print(f'completed processing {file.name}')    

            # create DataFrame and output to CSV
            df = pd.DataFrame(tracks)
            df.to_csv(f'../data/TLE/processed/{file.name[:-4]}.csv')
    


# Inital Validation

In [ ]:
initial_validate_tle_files(source_files)